In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
import re
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
top_words =5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

In [3]:
x_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [4]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print([id2word.get(i, " ") for i in x_train[6]])
print("                              --------------------------------------------------")
print(y_train[6])

['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'overly', 'to', 'descent', 'people', 'time', 'very', 'bland']
       

In [5]:
print("Maximum review length :", len(max((x_train + x_test), key=len)))
print("**************************")
print("Manimum review length :", len(min((x_train + x_test), key=len)))

Maximum review length : 2697
**************************
Manimum review length : 70


In [6]:
max_review_len = 500
X_train = sequence.pad_sequences(x_train, maxlen= max_review_len)
X_test = sequence.pad_sequences(x_test, maxlen= max_review_len)

In [7]:
x_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [8]:
X_train[45]

array([3570,   15,    2,    4,    2,    2,    2,    7,    2,    5,    2,
         21,    2,   47,    6, 1003,    4,    2,   47,    6,    2,   63,
          2,    2,    2, 2672,    8,    4,  311, 2009,  159,   59,    2,
         11, 2151,   10,   10,    2,   47,  115,  825,   17,   36,  135,
         32,    4,   96,   19,    6,  430,   10,   10,   33,    6, 1073,
         75,   67,    6, 1405, 1459,    7,    4,    2,  383, 2165, 3303,
         41,   97,   56,   11,    6, 3868, 2915,    2,    9,    6,    2,
          2, 2151,  214,    8,  124,    2, 1405,  137,   41, 4859, 2010,
          4, 1093,    2,  483,  137,    4,    2,  271,  143,    4,    2,
          7, 4576, 4137,   18,    2, 2119,   59,  214,    2,   19,    4,
       1093, 3878,   54,   57, 2066,  267,   11,    4,  598,   91,    2,
        136,    4, 1093, 3878, 4443,    8,  202,    4,    2,    6,  508,
         11, 2328,   41,    2,  143,    4, 4908,  511,    7,    2,    2,
          2,  494, 1013,    8,   40,   41,   21,   

In [7]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(5000, 32, input_length= 500))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation= "sigmoid"))
model.compile(loss= "binary_crossentropy", optimizer= "adam", metrics=["accuracy"])
print(model.summary())
model.fit(X_train, y_train, validation_data= (X_test, y_test), epochs= 20, batch_size= 64) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 3

In [6]:
model.evaluate(X_test, y_test)


NameError: name 'model' is not defined

In [11]:
model.save("sentiment_analysis_by_ashish.h5")

NameError: name 'model' is not defined

In [12]:
model_1 = load_model('sentiment_analysis_by_ashish.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [17]:
model.predict(X_test)

array([[0.01215553],
       [0.9999776 ],
       [0.09546384],
       ...,
       [0.03154647],
       [0.2164666 ],
       [0.9996847 ]], dtype=float32)

In [8]:
model_1.evaluate(X_test, y_test)

25000/25000 [==============================] - 134s 5ms/step


[0.6029752405941486, 0.8616799712181091]

In [9]:
model_1.evaluate(X_train, y_train)

25000/25000 [==============================] - 100s 4ms/step


[0.025763228882551194, 0.9933599829673767]